## Mở đầu

**Chủ đề:** Nén và phát hiện chuỗi bất thường bằng Trie và phân tích thống kê

**Sinh viên:** Trần Tấn Phát, Nguyễn Anh Quân

***Colab notebook này phục vụ cho việc demo chương trình.***

## Thiết lập

**[Cần chỉnh sửa lệnh git clone khi merge về master]**

In [ ]:
# Clone repo
!git clone --branch readme-colab https://github.com/quannguyen2412899/SCAD-TSA.git
%cd SCAD-TSA

In [ ]:
# Cài đặt Graphviz và python pandas
!apt-get install graphviz
%pip install graphviz pandas

## Biên dịch

In [ ]:
!mkdir -p bin
# Biên dịch module preprocess
!g++ -std=c++17 -I./include src/preprocess.cpp src/Preprocessor.cpp -o bin/preprocess
# Biên dịch module analyze
!g++ -std=c++17 -I./include src/analyze.cpp src/Analysis.cpp src/StatTrie.cpp -o bin/analyze
# Biên dịch module visualize
!g++ -std=c++17 -I./include src/visualize.cpp -o bin/visualize
# Biên dịch chương trình chính
!g++ -std=c++17 -I./include src/main_pipeline.cpp -o main_pipeline

## Chuẩn bị dữ liệu


In [ ]:
# Chuẩn bị thư mục
!mkdir data results

In [ ]:
import random
import string

# Cấu hình file output
output_file = "data/sample_generated.log"

print("🔄 Đang sinh dữ liệu giả lập...")

with open(output_file, "w") as f:
    # ---------------------------------------------------------
    # 1. DỮ LIỆU BÌNH THƯỜNG (NORMAL TRAFFIC)
    # ---------------------------------------------------------
    normal_patterns = [
        "INFO: GET /index.html 200",
        "INFO: GET /home.php 200",
        "INFO: GET /img/logo.png 200",
        "INFO: GET /img/banner.jpg 200",
        "INFO: POST /login 200"
    ]

    # Sinh 5000 dòng bình thường
    for _ in range(5000):
        f.write(random.choice(normal_patterns) + "\n")

    # ---------------------------------------------------------
    # 2. BẤT THƯỜNG TẦN SUẤT (FREQUENCY ANOMALY)
    # - Đặc điểm: Chuỗi duy nhất, chưa từng thấy.
    # ---------------------------------------------------------
    # Kẻ tấn công thử lệnh shutdown hệ thống (chỉ 1 lần)
    f.write("CRITICAL: SYSTEM SHUTDOWN INITIATED BY REMOTE HOST\n")
    print("-> Gài 1 Bất thường Tần suất (Shutdown Command).")

    # ---------------------------------------------------------
    # 3. BẤT THƯỜNG ĐỘ DÀI (LENGTH ANOMALY - Rare Length)
    # - Đặc điểm: Độ dài mà 5000 dòng kia không có.
    # ---------------------------------------------------------
    # Tấn công Buffer Overflow: Chuỗi dài 500 ký tự 'A'
    buffer_overflow = "ERROR: Input buffer overflow " + ("A" * 500)
    f.write(buffer_overflow + "\n")
    print("-> Gài 1 Bất thường Độ dài (Buffer Overflow - 500 chars).")

    # ---------------------------------------------------------
    # 4. BẤT THƯỜNG ENTROPY CAO (ENTROPY ANOMALY)
    # - Đặc điểm: Tại một node, nhánh rẽ ra quá nhiều hướng (Hỗn loạn).
    # ---------------------------------------------------------
    # Tấn công Fuzzing: Thử 100 ký tự rác khác nhau vào tham số 'token='
    fuzzing_chars = string.ascii_letters + string.digits + "!@#$%^&*()"
    for i in range(100):
        # Chọn ngẫu nhiên 1 ký tự để tạo nhánh ngay lập tức
        char = fuzzing_chars[i % len(fuzzing_chars)]
        f.write(f"WARN: Invalid token={char}...\n")

    print("-> Gài 100 dòng Bất thường Entropy Cao (Fuzzing Attack).")

print(f"✅ Đã tạo file: {output_file}")

## Chạy thử chương trình

#### Chạy pipeline

In [ ]:
# Phân tích dữ liệu sample, hiển thị cây trie thu gọn, chọn phần vị tần suất, độ dài và entropy lần lượt là P1, P1, P99
!./main_pipeline data/sample_generated.log results --visual-partial --perc-freq=1 --perc-len=1 --perc-entropy=99

#### Hiển thị kết quả
##### Báo cáo tổng quan

In [ ]:
!cat results/overall_report.txt

##### Danh sách bất thường

In [ ]:
# Danh sách bất thường tần suất
import pandas as pd
df = pd.read_csv("results/frequency_anomalies.csv")
df.index = df.index + 1
display(df)

In [ ]:
# Danh sách bất thường độ dài
import pandas as pd
df = pd.read_csv("results/length_anomalies.csv")
df.index = df.index + 1
display(df)

In [ ]:
# Danh sách bất thường entropy
import pandas as pd
df = pd.read_csv("results/entropy_anomalies.csv")
df.index = df.index + 1
display(df)

Ngoài ra còn có danh sách tất cả các chuỗi/tiền tố được thống kê trong ```results/all_entries.csv```

In [ ]:
import pandas as pd
df = pd.read_csv("results/all_entries.csv")
df.index = df.index + 1
display(df)

##### Hiển thị cây Trie

In [ ]:
from IPython.display import Image, display
display(Image("results/partial_trie.png"))

## Chạy thử trên dữ liệu tuỳ ý

Chuẩn bị dữ liệu trên Google drive và chạy theo hướng dẫn:

In [ ]:
from google.colab import drive

# Mount drive vào thư mục /content/drive
drive.mount('/content/drive')

print("\n✅ Đã kết nối Drive thành công!")

Tuỳ chỉnh cấu hình:

In [ ]:
import os

# Dán đường dẫn file log của bạn vào đây (ví dụ: /content/drive/MyDrive/uni/251/DSA/SCAD-TSA/raw_install_log.txt):
user_file_path = ""

# Cấu hình Tiền xử lý (Preprocess)
# Để trống nếu muốn dùng mặc định. Nhập Regex nếu file có format phức tạp.
custom_regex = ""   # ví dụ: r"^\S+\s+\S+\s+\S+\s(.+)$"
custom_delim = ""
custom_ignore = ""

# Cấu hình Phân tích (Analyze)
# Chọn ngưỡng phần vị (Percentile) để phát hiện bất thường:
perc_freq = 1
perc_len = 1
perc_entropy = 99

# Trực quan hóa
# param ["Partial (Anomalies Only)", "Complete Trie (Caution!)", "Frequency Only", "Entropy Only"]
visual_mode = "Partial (Anomalies Only)"

# --- VALIDATION LOGIC ---
if not os.path.exists(user_file_path):
    print(f"❌ LỖI: Không tìm thấy file tại '{user_file_path}'")
    print("👉 Vui lòng kiểm tra lại đường dẫn.")
else:
    print(f"✅ Đã tìm thấy file: {os.path.basename(user_file_path)}")
    print("Vài dòng đầu của file (nếu muốn sử dụng regex):")
    get_ipython().system("!head -n 5 "+user_file_path)

Chạy pipeline:

In [ ]:
# Thực thi Pipeline
import subprocess

# Map Visual Mode sang Flag
visual_flags_map = {
    "Partial (Anomalies Only)": "--visual-partial",
    "Complete Trie (Caution!)": "--visual-complete",
    "Frequency Only": "--visual-freq",
    "Entropy Only": "--visual-entropy"
}
selected_visual_flag = visual_flags_map[visual_mode]

# Xây dựng câu lệnh
cmd_parts = [
    "./main_pipeline",
    f'"{user_file_path}"',  # Input file
    "results_custom",       # Output folder
    selected_visual_flag,   # Visual flag
    f"--perc-freq={perc_freq}",
    f"--perc-len={perc_freq}",
    f"--perc-entropy={perc_entropy}"
]

# Thêm các cờ Preprocess tùy chọn
cmd_parts.append(f'--regex="{custom_regex}"')
cmd_parts.append(f'--delim="{custom_delim}"')
cmd_parts.append(f'--ignore="{custom_ignore}"')

# Ghép thành chuỗi lệnh hoàn chỉnh
full_command = " ".join(cmd_parts)

print(f"🚀 Đang chạy lệnh: {full_command}\n")

# Thực thi
!mkdir results_custom
get_ipython().system(full_command)

print("\n✅ Phân tích hoàn tất! Xem kết quả bên dưới.")

Hiển thị báo cáo tổng quan

In [ ]:
!cat results_custom/overall_report.txt

Hiển thị các danh sách:

In [ ]:
# Danh sách bất thường tần suất
import pandas as pd
df = pd.read_csv("results_custom/frequency_anomalies.csv")
df.index = df.index + 1
display(df)

In [ ]:
# Danh sách bất thường độ dài
import pandas as pd
df = pd.read_csv("results_custom/length_anomalies.csv")
df.index = df.index + 1
display(df)

In [ ]:
# Danh sách bất thường entropy
import pandas as pd
df = pd.read_csv("results_custom/entropy_anomalies.csv")
df.index = df.index + 1
display(df)

In [ ]:
import pandas as pd
df = pd.read_csv("results_custom/all_entries.csv")
df.index = df.index + 1
display(df)

Hiển thị hình ảnh trực quan Trie:

In [ ]:
from IPython.display import Image, display
display(Image("results_custom/partial_trie.png"))